# Création de la base des fausses réactions à partir du recommender

In [ ]:
#Bibliothèques dont on a besoin
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow.keras as keras

In [ ]:
#Pour gérer les progress bar
import tqdm.notebook as tqdm

In [ ]:
#Importations de AiZynthFinder
from aizynthfinder.chem import Molecule, Reaction, MoleculeException
from aizynthfinder.training.utils import (
    Config,
    create_reactants_molecules,
    reverse_template,
    reaction_hash,
    reactants_to_fingerprint,
)
from aizynthfinder.utils.models import CUSTOM_OBJECTS, load_keras_model


In [ ]:
#On récupère les bases de données adéquates
templates_all = pd.read_csv("Purified_Templates.csv")
reactions_data = pd.read_json("uspto-reactions.json")
reactions_fp_data = pd.read_csv('Reactions_Fingerprint_True.csv')

In [ ]:
#On récupère notre modèle déjà entraîné
model = keras.models.load_model("our_recommender_model_2")

In [ ]:
def fp_from_fp_db(fp_db):
    '''
    Renvoie le fingerprint à partir de sa forme stockée (indices des valeurs non nulles etc...) 
    sous forme de tableau de 2048 éléments
    '''
    res = np.zeros(2048)
    list_idx = fp_db.split(" ")
    for el in list_idx[:-1]:
        lfp = el.split("-")
        n = len(lfp)
        if n == 1:
            res[int(lfp[0])] = 1
        elif n==2:
            res[int(lfp[0])] = int(lfp[-1])
        else:
            res[int(lfp[0])] = -int(lfp[-1])
    return res

In [ ]:
def recommended_rows(row1,row2):
    '''
    Renvoie le tableau des 20 templates les plus probables entre row1 et row2 
    '''
    fingerprints = np.array([fp_from_fp_db(reactions_fp_data.iloc[row,1])- fp_from_fp_db(reactions_fp_data.iloc[row,2]) for row in range(row1,row2)])
    return np.argsort(model.predict(fingerprints))[::-1][:20] #On récupère les plus 20 plus probables

def predictions(row1,row2):
    '''
    Ne garde que les bonnes prédictions et stocke les mauvaises dans skip sous forme d'un indice
    '''
    fingerprints = []
    skip = []
    for row in range(row1,row2):
        try : 
            fingerprints+= [fp_from_fp_db(reactions_fp_data.iloc[row,1])- fp_from_fp_db(reactions_fp_data.iloc[row,2])]
        except:
            skip.append(row)
            fingerprints += [np.zeros(2048)]
    fingerprints = np.array(fingerprints)
    predictions = model.predict(fingerprints)
    return [np.argsort(predictions[i])[::-1][:20] for i in range(fingerprints.shape[0])], skip
    

In [ ]:
pred,skip = predictions(0,10000)

In [ ]:
smarts_templates = templates_all[:200000].drop_duplicates(subset ='reaction_smarts')['reaction_smarts'].to_list()
my_map = dict(list(enumerate(smarts_templates)))

In [ ]:
inv_map = {v: k for k, v in my_map.items()}

In [ ]:
def recommender_sample_lib(r1,r2):
    '''
    Retourne un dataset contenant des fausses réactions obtenues à l'aide du recommender
    '''
    data_frame = pd.DataFrame(columns=['Reactifs', 'Produits'])
    #On ne garde que les réactions qui ont donné quelque chose 
    set_val = (set(list(range(r1,r2)))-set(skip))
    
    #Progress Bar
    p_bar = tqdm.tqdm(total = len(set_val))
    for val in set_val:
        if val in skip:
            continue
        row_reactants = reactions_data.iloc[val,1]
        row_products  = reactions_data.iloc[val,2]
        #Si on a bien une réaction faisable
        if (type(row_reactants) == str):
             mols = create_reactants_molecules(row_reactants)
        else:
             continue
        try:
            #On élimine les éventuelles erreurs liées à la sanitization (normalisation) des molécules
            ref_mol = Molecule(smiles=row_products, sanitize=True)
        except MoleculeException:
             continue
        new_product=None
        for template_row in pred[val-r1]:
            smarts_fwd = reverse_template(my_map[template_row])
            try:
                new_product = Reaction(mols=mols, smarts=smarts_fwd).apply()[0][0]
            except (ValueError, IndexError):
                continue
            #Si le produit obtenu est identique à celui de base, on ne garde pas la réaction
            if new_product.basic_compare(ref_mol):
                continue
            break  # Si on arrive jusqu'ici, c'est que la réaction obtenue a de grandes chances d'être fausse
        if not new_product:
            continue
        #On ajoute la ligne au data_frame    
        data_frame = pd.concat([pd.DataFrame(data={"Reactifs" : row_reactants, "Products" : [new_product.smiles]}),data_frame], ignore_index=True)
        p_bar.update(1)
               
    data_frame.to_csv(f"Data_frame_{r1}.csv")
    return data_frame

In [ ]:
#Pour que ca aille plus vite, on séquence en plusieurs morceaux
l2 = []
p_b = tqdm.tqdm(total=20)
for i in range(21,40):
    pred,skip = predictions(10000*i,10000*(i+1))
    l2.append(recommender_sample_lib(10000*i,10000*(i+1)))
    p_b.update(1)

In [ ]:
data_final = pd.concat(l2,ignore_index=True)

In [ ]:
#On enregistre en CSV
data_final.to_csv('Data_False_Reactions_Recommender.csv')